In [5]:
import pandas as pd
import numpy as np
df = pd.read_csv("usps-2cls.csv", header= None)

df.head()

,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,256
0,0,0,95,107,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,78,95,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,42,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [492]:
class Bayes:
    def __init__(self):
        pass
    def fit(self,X,y):
        classes = np.unique(y)
        cls_dict={k:{} for k in classes}
        for cls in classes:
            idx = np.where(y == cls)
            x_cls = X[idx]
            mu = x_cls.mean(axis=0)
            cov = np.diag(np.ones(X.shape[1]))
            d = {"mu":mu,"cov":cov}
            cls_dict[cls].update(d)
        self.cls_dict = cls_dict
    @staticmethod
    def _qda(x, mu, cov, prior):
        cv_inv = np.linalg.inv(cov)
        a = np.dot(np.dot(x.T,(-.5*cv_inv)),x)
        b = np.dot(np.dot(cv_inv,mu).T,x)
        c = -.5*np.dot(np.dot(mu.T,cv_inv),mu)
        d = -.5*np.log(np.linalg.det(cov))
        #e = np.log(prior)
        return a+b+c+d#+e
    def predict(self,x, prior):
        best_score =float("inf")
        scores = {}
        for k,v in self.cls_dict.items():
            mu = v["mu"]
            cov = v["cov"]
            score = self._qda(x, mu, cov, prior)
            scores.update({k:score})
        sum_scores = sum([v for k,v in scores.items()])
        for k,v in scores.items():
            score = (v/sum_scores)
            if score < best_score:
                cls = k
                best_score = score
        return cls
        
    
            

In [493]:
X=df.iloc[:,:-1].to_numpy()
y = df.iloc[:,-1].to_numpy()

In [494]:
bayes = Bayes()

model = bayes.fit(X=X, y = y)

In [495]:
r = []
for i in range(X.shape[0]):
    x = X[i]
    r.append(bayes.predict(x,.5))

In [496]:
from sklearn.metrics import accuracy_score

accuracy_score(y, r)

0.9281818181818182

In [268]:
mu = bayes.cls_dict[0]["mu"]
cov = bayes.cls_dict[0]["cov"]
prior = .5
x = X[0]


r1 = x.T*(-1/2*cov**-1)*x+(cov**-1*mu).T*x-1/2*mu.T*cov**-1*mu-1/2*np.log(np.abs(cov)) + np.log(prior)
mu = bayes.cls_dict[1]["mu"]
cov = bayes.cls_dict[1]["cov"]
r2 = x.T*(-1/2*cov**-1)*x+(cov**-1*mu).T*x-1/2*mu.T*cov**-1*mu-1/2*np.log(np.abs(cov)) + np.log(prior)
r3 = r1+r2
r3 == sum([r1,r2])

/home/jfftilton/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in reciprocal
  import sys
/home/jfftilton/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in multiply
  import sys
/home/jfftilton/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in add
  import sys
/home/jfftilton/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys
/home/jfftilton/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in reciprocal
  # Remove the CWD from sys.path while we load stuff.
/home/jfftilton/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in multiply
  # Remove the CWD from sys.path while we load stuff.
/home/jfftilton/.local/lib/python3.6/site-packages/ipykernel_

array([[ True, False, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       ...,
       [False, False, False, ...,  True, False, False],
       [False, False, False, ..., False,  True, False],
       [False, False, False, ..., False, False,  True]])

In [290]:
a = x.T*(-1/2*np.linalg.inv(cov))*x
b = (np.linalg.inv(cov)*mu).T*x
c=1/2*mu.T*np.linalg.inv(cov)*mu
d =1/2*np.log(np.linalg.slogdet(cov))

a+b-c

/home/jfftilton/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


array([[-2.14214876e-03,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -3.44450000e-01,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -4.09899273e+03, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -9.97014215e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00, -3.03137293e+01,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00, -9.26846942e+00]])

In [419]:
def _qda(x, mu, cov, prior):
    a = -.5* np.linalg.det(cov)
    b = -.5*(x-mu).T
    c = x-mu
    d = np.log(prior)
    e = np.dot(np.dot(b,np.linalg.inv(cov)),c)

    return a+e+d

cv = np.diag(np.ones(cov.shape[0]))

In [413]:
_qda(x, mu, c, prior)

array([[-1413192.82335545, -1407654.36294718, -1381377.95744305, ...,
        -1381318.68735544, -1357261.61735379, -1382477.74684636],
       [-1407654.36294718, -1402190.72234718, -1376269.28834718, ...,
        -1376210.81894718, -1352478.73914718, -1377354.22054718],
       [-1381377.95744305, -1376269.28834718, -1352031.96409924, ...,
        -1351977.29344305, -1329787.08154388, -1353046.4084976 ],
       ...,
       [-1381318.68735544, -1376210.81894718, -1351977.29344305, ...,
        -1351922.63135545, -1329735.89735379, -1352991.57884635],
       [-1357261.61735379, -1352478.73914718, -1329787.08154388, ...,
        -1329735.89735379, -1308960.80331247, -1330736.83262652],
       [-1382477.74684636, -1377354.22054718, -1353046.40849759, ...,
        -1352991.57884635, -1330736.83262652, -1354063.80313726]])

In [452]:
mu = bayes.cls_dict[0]["mu"]
cv_inv = np.linalg.inv(cv)
def _qda(x, mu, cov, prior):
    cv_inv = np.linalg.inv(cv)
    a = np.dot(np.dot(x.T,(-.5*cv_inv)),x)
    b = np.dot(np.dot(cv_inv,mu).T,x)
    c = -.5*np.dot(np.dot(mu.T,cv_inv),mu)
    d = -.5*np.log(np.linalg.det(cv))
    e = np.log(prior)
    return a+b+c+d+e



AttributeError: 'Bayes' object has no attribute 'cls_dict'

In [ ]:
cv_inv = np.linalg.inv(cv)
cv_det

In [450]:
d

-0.0

In [394]:
s = np.diag(np.ones(X.shape[1]))
mu.T*np.linalg.inv(s)*x-1/2*mu.T*np.linalg.inv(s)*mu+np.log(.5)

array([[ -0.69528933,  -0.69314718,  -0.69314718, ...,  -0.69314718,
         -0.69314718,  -0.69314718],
       [ -0.69314718,  -1.03759718,  -0.69314718, ...,  -0.69314718,
         -0.69314718,  -0.69314718],
       [ -0.69314718,  -0.69314718, 412.81412183, ...,  -0.69314718,
         -0.69314718,  -0.69314718],
       ...,
       [ -0.69314718,  -0.69314718,  -0.69314718, ..., -10.66328933,
         -0.69314718,  -0.69314718],
       [ -0.69314718,  -0.69314718,  -0.69314718, ...,  -0.69314718,
        -31.00687652,  -0.69314718],
       [ -0.69314718,  -0.69314718,  -0.69314718, ...,  -0.69314718,
         -0.69314718,  -9.9616166 ]])

In [401]:
np.dot(np.dot(mu,np.linalg.inv(s)),x)

array([[1406904.30090909]])

In [393]:
mu = mu.reshape(1,mu.shape[0])

In [391]:
x = x.reshape(x.shape[0],1)

In [309]:
c1 = np.array([np.array([1.679,-.0461]), np.array([-.0461,1.5985])])
c2 = np.array([np.array([2.0114,-.3334]), np.array([-.3334,1.791])])
mu1 = np.array([-.4038,-.1937])
mu2 = np.array([.7533,.3613])

prior1 = .651
prior2 = .349


def qda(x, mu,cov,prior):
    return -1/2*np.log(np.linalg.det(cov)) -1/2*(x-mu).T*np.linalg.inv(cov)*(x-mu)+np.log(prior)

x = np.array([-2,-2])


q1 = qda(x, mu1.T,c1,prior1)
q2 = qda(x, mu2.T,c2,prior2)

In [312]:
q1

array([[-1.68182452, -0.95052504],
       [-0.94438069, -1.94384703]])

In [270]:
cov = bayes.cls_dict[1]["cov"]
np.linalg.det(cov)

1.0

In [280]:
mu = bayes.cls_dict[0]["mu"]
cov = bayes.cls_dict[0]["cov"]
d1 = np.log(prior)-1/2*np.log(np.abs(cov))-1/2*(x-mu).T*cov**-1*(x-mu)

mu = bayes.cls_dict[1]["mu"]
cov = bayes.cls_dict[1]["cov"]
d2 = np.log(prior)-1/2*np.log(np.abs(cov))-1/2*(x-mu).T*cov**-1*(x-mu)

d_tot = d1+d2

(d2/d_tot)

/home/jfftilton/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
/home/jfftilton/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in reciprocal
  This is separate from the ipykernel package so we can avoid doing imports until
/home/jfftilton/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in subtract
  This is separate from the ipykernel package so we can avoid doing imports until
/home/jfftilton/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys
/home/jfftilton/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in reciprocal
  import sys
/home/jfftilton/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: Runt

array([[0.01151749,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan, 0.00231802,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan, 0.02912195, ...,        nan,        nan,
               nan],
       ...,
       [       nan,        nan,        nan, ..., 0.67723578,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan, 0.71235137,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
        0.77359888]])

In [272]:
r1.shape

(256, 256)

In [273]:
r3 = -1/2*np.log(np.abs(cov))-1/2*(x-mu).T*cov**-1*(x-mu)+np.log(prior)

/home/jfftilton/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.
/home/jfftilton/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in reciprocal
  """Entry point for launching an IPython kernel.
/home/jfftilton/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in subtract
  """Entry point for launching an IPython kernel.


In [274]:
scores = bayes.predict(x,.5)
sum_scores = sum([v for k,v in scores.items()])
for k,v in scores.items():
    score = v/sum_scores
    print(np.sum(score))

AttributeError: 'numpy.int64' object has no attribute 'items'

In [275]:
scores[1].mean()

IndexError: invalid index to scalar variable.

In [360]:
r = []
for i in range(X.shape[0]):
    x = X[i]
    r.append(model.predict(x,.5))

AxisError: axis 1 is out of bounds for array of dimension 1

In [361]:
model.predict(X,.5)

array([array([0, 1]), array([0, 1]), array([0, 1]), ...,
       array([0, 0, 0, ..., 1, 1, 1]), array([0, 0, 0, ..., 1, 1, 1]),
       array([0, 0, 0, ..., 1, 1, 1])], dtype=object)

In [278]:
from sklearn.metrics import accuracy_score

accuracy_score(y, r)

0.41045454545454546

In [10]:
np.cov(df[df.iloc[:,-1]==0])

(1100, 1100)

In [11]:
y = df.iloc[:,-1]

In [13]:
idx=np.where(y == 1)

In [18]:
X = df.iloc[:,:-1].to_numpy()

In [21]:
X[idx].mean(axis=1)

array([73.5078125 , 56.76171875, 48.70703125, ..., 59.390625  ,
       69.6796875 , 76.65234375])

In [319]:
from scipy.stats import multivariate_normal
idx = np.where(y == 1)
x_cls = X[idx]
mvn = multivariate_normal.pdf(x_cls, mean =x_cls.mean(axis=0))

In [322]:
d=np.array([1,2,3])
d.argmax(1)

AxisError: axis 1 is out of bounds for array of dimension 1